In [ ]:
import torch
import torchaudio
import sys

from artst.tasks.artst import ArTSTTask
from artst.models.artst import ArTSTTransformerModel
from sentencepiece import SentencePieceProcessor

from fairseq import checkpoint_utils, utils


checkpoint = torch.load('.../ArTST-hf/MGB2_ASR.pt')  # path to change
checkpoint['cfg']['task'].t5_task = 's2t' # or "s2t" for asr
checkpoint['cfg']['task'].data = '.../ArTST-hf'  # path to change
task = ArTSTTask.setup_task(checkpoint['cfg']['task'])
task.args.bpe_tokenizer = '.../ArTST-hf/asr_spm.model'  # path to change

model = ArTSTTransformerModel.build_model(checkpoint['cfg']['model'], task)
model.load_state_dict(checkpoint['model'])

In [ ]:
# lms, _ = checkpoint_utils.load_model_ensemble([lm_path], arg_overrides=overrides, task=None)
lms = [None]
lm_weight = None
models = [model]

extra_gen_cls_kwargs = {"lm_model": lms[0], "lm_weight": lm_weight}
generator = task.build_generator([model], task.args, extra_gen_cls_kwargs=extra_gen_cls_kwargs)

source, sr = torchaudio.load('....')
padding_mask = (torch.BoolTensor(source.shape).fill_(False))

assert sr == 16000
sample = {
    "net_input": {
        "source": source,
        "padding_mask": padding_mask,
        "task_name": "s2t",
    }
}
hypos = task.inference_step(
    generator,
    models,
    sample
)

In [ ]:
def get_symbols_to_strip_from_output(generator):
    if hasattr(generator, "symbols_to_strip_from_output"):
        return generator.symbols_to_strip_from_output
    else:
        return {generator.eos}

def decode_fn(x):
    if bpe is not None:
        x = bpe.decode(x)
    if tokenizer is not None:
        x = tokenizer.decode(x)
    return x

tokenizer = None
bpe = task.build_bpe(task.args)

hypo_tokens, hypo_str, alignment = utils.post_process_prediction(
    hypo_tokens=hypos[0][0]["tokens"].int().cpu(),
    src_str="",
    alignment=hypos[0][0]["alignment"],
    align_dict=None,
    tgt_dict=task.dicts['text'],
    remove_bpe=None,
    extra_symbols_to_ignore=get_symbols_to_strip_from_output(generator)
)
decode_fn(hypo_str)